# Chapter16. 코루틴
- 코루틴으로 작동하는 제너레이터의 동작과 상태
- 데커레이터를 이용해서 코루틴을 자동으로 기동하기
- 제너레이터 객체의 close()와 throw() 메서드를 통해 호출자가 코루틴을 제어한ㄴ 방법
- 종료할 때 코루틴이 값을 반환하는 방법
- 새로운 yield from 구문의 사용법과 의미
- 사용 예: 시뮬레이션의 동시 활동을 관리하기 위한 코루틴

## 16.1 코루틴은 제너레이터에서 어떻게 진화했는가?
(코루틴 기반 구조는 파이썬 2.5에 구현된 PEP342 - https://peps.python.org/pep-0342/) 에 설명되어 있다. 이때부터 yield 키워드를 표현식에 사용할 수 있게 되었으며, send()a메서드가 제너레이터 API에 추가되었다. 제너레이터의 호출자는 send()를 이용해서 제너레이터 함수 내부의 yield 표현식의 값이 될 데이터를 전송할 수 있다. 이렇게 제너레이터가 호출자에 데이터를 생성해주고 호출자로부터 데이터를 받으면서 호출자와 협업하는 프로시저인 코루틴이 된다.

최근 코루틴으로의 혁신적인 진화는 파이썬 3.3(2012)에서 구횐된 PEP380 [하위 제너레이터에 위임하기 위한 구문 제안서](https://peps.python.org/dev/peps/pep-0380/)에 기술되어 있다. PEP380은 제너레이터 함수에 다음과 같이 두 가지 구문 변경을 정의해서 훨씬 더 유용하게 코루틴으로 사용할 수 있도록 만들었다.

- Procedure는 데이터베이스에서 일련의 쿼리들을 하나의 논리적인 작업으로 실행하기 위한 기술이며, 이는 데이터베이스의 특정 기능이나 작업을 자동화하고 자주 사용되는 쿼리들을 하나의 함수처럼 실행하기 위한 쿼리의 집합으로 구성되어 있다. 특히, 이 쿼리들은 데이터베이스 개발자가 일괄적인 작업을 수행하거나 특정한 목적을 위해 사용할 수 있도록 미리 구성되어 있다.